In [3]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, caseless, normalized, dictionary,  gte, lte
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.relations import gnc_relation
import gzip
from dataclasses import dataclass
from typing import Iterator, Optional
from tqdm import tqdm 
from typing import List


In [4]:
# Define Yargy rules for extracting information
TITLES = morph_pipeline(["science", "style", "culture", "life", "economics", "business", "travel", "forces", "media", "sport"])
SURNAME = or_(
    rule(gram("Surn")),
    rule(and_(is_capitalized(), gram("NOUN"))),
    rule(and_(is_capitalized(), gram("ADJF"))),
    rule(and_(is_capitalized(), gram("UNKN"))),
    rule(and_(is_capitalized(), gram("ADVB")))
)

JUST_NAME = or_(
    rule(gram("Name")),
    rule(gram("Name"), "-", gram("Name"))
)

NAME = or_(
    rule(JUST_NAME, SURNAME.optional()),
    rule(SURNAME, JUST_NAME.optional())
)

BIRTH_VERB = morph_pipeline(["родился", "появился на свет"])

BIRTH_PLACE_ABBR = rule(gram("PREP").optional(), gram("Abbr"))

BIRTH_PLACE_TAG = morph_pipeline([
    "штат", "город", "столица", "деревня", "окраина", "север", "юг",
    "восток", "запад", "край", "область", "республика", "область", "страна"
])

BIRTH_PLACE_FULL = rule(
    BIRTH_PLACE_TAG.optional(),
    or_(
        and_(is_capitalized(), gram("ADJF")),
        and_(is_capitalized(), gram("NOUN"))
    ).repeatable(),
    BIRTH_PLACE_TAG.optional()
)

BIRTH_PLACE = rule(
    gram("PREP"),
    or_(BIRTH_PLACE_FULL.repeatable(), BIRTH_PLACE_ABBR)
)

DAY = and_(gte(1), lte(31))

MONTHS = dictionary({
    'январь', 'февраль', 'март', 'апрель', 'мая', 'июнь',
    'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь'
})

YEAR = and_(gte(1), lte(2024))

YEAR_WORDS = morph_pipeline(["г.", "год"])

DATE = or_(
    rule(YEAR, YEAR_WORDS.optional()),
    rule(DAY, MONTHS, YEAR, YEAR_WORDS.optional()),
    rule(DAY, MONTHS)
)

BIRTH_DATE = rule(gram("PREP").optional(), DATE)

BIRTH_PLACE_FULL = rule(
    BIRTH_PLACE_ABBR.optional(),
    BIRTH_PLACE_FULL  
)

S = rule(
    NAME,
    BIRTH_VERB,
    or_(
        rule(BIRTH_PLACE.repeatable(), BIRTH_DATE),
        rule(BIRTH_DATE, BIRTH_PLACE.repeatable())
    )
)

# Initialize Yargy parser with the defined rule
text = "Чжан родилась 22 июля 2013 года в Сиане и работает в Уси."  
text = text.replace(",", "")
import re
text = re.sub(r'\(.*?\)', '', text)
parser = Parser(S)

@dataclass
class Text:
    label: str
    title: str
    text: str

@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

def read_texts(fn: str) -> List[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        return [Text(*line.strip().split("\t")) for line in f]

texts = read_texts("news.txt.gz")
entries = []

def extract_entries(texts: List[Text]) -> List[Entry]:
    for text in tqdm(texts, disable=True):
        text = text.text.replace(",", "")
        text = re.sub(r'\(.*?\)', '', text)
        try:
            for match in parser.findall(text):
                birth_date_parts = match.fact.BIRTH_DATE.tokens
                birth_date = " ".join(part.value for part in birth_date_parts)
                entry = Entry(
                    name=match.fact.NAME.normalized,
                    birth_date=birth_date,
                    birth_place=match.fact.BIRTH_PLACE.normalized if match.fact.BIRTH_PLACE else None,
                )
                entries.append(entry)
        except Exception as e:
            print(f"Error processing text: {text}, Error: {e}")
    return entries

# Run the extraction
result_entries = extract_entries(texts)
for result_entry in result_entries:
    print(result_entry)


Error processing text: Голливудская актриса Бетси Палмер  известная по фильму ужасов «Пятница 13-е» умерла в хосписе города Данбери штат Коннектикут сообщает Variety. Ей было 88 лет. Смерть наступила от естественных причин.Актриса также активно играла в театре и участвовала в телевизионных шоу. На ее счету 19 фильмов и 22 телепередачи. Тем не менее широкую известность ей принесли именно съемки в хорроре «Пятница 13-е» в 1980 году. Она воплотила на экране образ матери злодея Джейсона Вурхиза — миссис Памелы Вурхиз — и стала главным отрицательным персонажем фильма. Актриса приняла участие и в сиквеле картины который вышел спустя год. Изначально она намеревалась отказаться от съемок сочтя такую ленту недостойной ее внимания но согласилась из-за финансовых затруднений.В дальнейшие годы Палмер продолжала сниматься на телевидении. Она получила небольшие роли в телесериалах «Она написала убийство» «Тихая пристань» и «Чарльз в ответе».Бетси Палмер родилась в 1926 году в США в семье выходца из 